In [15]:
# import
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import scipy
import math

In [14]:
# get the data
bet=pd.read_csv('^bet_d.csv')

In [16]:
# keep the giving priod of time
bet=bet[bet['Date']>='1997-09-19']
bet=bet[bet['Date']<='2012-05-21']
bet = bet.set_index('Date')
bet =pd.DataFrame(bet['Close'])

In [17]:
print(bet)

              Close
Date               
2000-10-31   528.94
2000-11-01   527.24
2000-11-02   528.65
2000-11-03   525.09
2000-11-06   521.20
...             ...
2012-05-15  4751.52
2012-05-16  4720.80
2012-05-17  4715.69
2012-05-18  4653.51
2012-05-21  4595.31

[2863 rows x 1 columns]


In [18]:
# calculate the daily logaritmic return
log_return_bet=np.log(bet/bet.shift(1))
# eliminate the firs position (because is blank)
log_return_bet=log_return_bet.iloc[1:]

In [24]:
# get the the lowest 1% data
q1=np.percentile(log_return_bet, 1)
print(len(log_return_bet[log_return_bet['Close']<q1]))

29


In [27]:
# estimate the regresion  ln[1-P(r<X)]=C -lln([X])+E
import statsmodels.distributions.empirical_distribution
x=np.log(np.abs(log_return_bet[log_return_bet['Close']<q1]['Close']))
y=list()
log_return_bet_q1=log_return_bet[log_return_bet['Close']<q1]
ecdf=statsmodels.distributions.empirical_distribution.ECDF(log_return_bet['Close'])
for i in range(0, len(log_return_bet_q1)):
    y.append(ecdf(log_return_bet_q1.iloc[i]['Close']))
y=np.log(y)
slope,intercept,r_value,p_value,std_err=stats.linregress(x,y)

In [30]:
# empirical probabilitys
print( len(log_return_bet[log_return_bet['Close']<-0.03])/len(log_return_bet)*100)
print( len(log_return_bet[log_return_bet['Close']<-0.06])/len(log_return_bet)*100)
print( len(log_return_bet[log_return_bet['Close']<-0.09])/len(log_return_bet)*100)


3.6338225017470296
0.628930817610063
0.2445842068483578


In [32]:
# estimated probability
print(math.exp(intercept)*(0.03**(slope))*100)
print(math.exp(intercept)*(0.06**(slope))*100)
print(math.exp(intercept)*(0.09**(slope))*100)

5.756788448972305
0.6838773375939697
0.19668430375894036


In [35]:
# empirical periodicity of this event (in years)
print(1/(250*len(log_return_bet[log_return_bet['Close']<-0.03])/len(log_return_bet)))
print(1/(250*len(log_return_bet[log_return_bet['Close']<-0.06])/len(log_return_bet)))
print(1/(250*len(log_return_bet[log_return_bet['Close']<-0.09])/len(log_return_bet)))

0.11007692307692307
0.636
1.6354285714285715


In [36]:
# estimated periodicity of this event (in years)
print(1/(250*math.exp(intercept)*(0.03**(slope))))
print(1/(250*math.exp(intercept)*(0.06**(slope))))
print(1/(250*math.exp(intercept)*(0.09**(slope))))

0.06948318555485698
0.5849002123791491
2.033715921176134
